In [41]:
from scipy.ndimage.filters import gaussian_laplace, minimum_filter

def localMinima(data, threshold):
    from numpy import ones, nonzero, transpose

    if threshold is not None:
        peaks = data < threshold
    else:
        peaks = ones(data.shape, dtype=data.dtype)

    peaks &= data == minimum_filter(data, size=(3,) * data.ndim)
    return transpose(nonzero(peaks))

def blobLOG(data, scales=range(1, 10, 1), threshold=-30.0):
    """Find blobs. Returns [[scale, x, y, ...], ...]"""
    from numpy import empty, asarray

    data = asarray(data)
    scales = asarray(scales)

    log = empty((len(scales),) + data.shape, dtype=data.dtype)
    for slog, scale in zip(log, scales):
        slog[...] = scale ** 2 * gaussian_laplace(data, scale)

    for slog, scale in zip(log, scales):
        # plt.title(scale)
        # plt.imshow(slog[slog.shape[0] // 2], vmin=np.min(log), vmax=np.max(log))
        # plt.colorbar()
        # plt.show()
        pass

#     plt.title(scale)
#     plt.imshow(data[data.shape[0] // 2])
#     plt.colorbar()
    # plt.show()

    peaks = localMinima(log, threshold=threshold)
    peaks[:, 0] = scales[peaks[:, 0]]
    return peaks

def sphereIntersection(r1, r2, d):
    # https://en.wikipedia.org/wiki/Spherical_cap#Application

    valid = (d < (r1 + r2)) & (d > 0)
    return (np.pi * (r1 + r2 - d) ** 2
            * (d ** 2 + 6 * r2 * r1
               + 2 * d * (r1 + r2)
               - 3 * (r1 - r2) ** 2)
            / (12 * d)) * valid

def circleIntersection(r1, r2, d):
    # http://mathworld.wolfram.com/Circle-CircleIntersection.html
    from numpy import arccos, sqrt

    return (r1 ** 2 * arccos((d ** 2 + r1 ** 2 - r2 ** 2) / (2 * d * r1))
            + r2 ** 2 * arccos((d ** 2 + r2 ** 2 - r1 ** 2) / (2 * d * r2))
            - sqrt((-d + r1 + r2) * (d + r1 - r2)
                   * (d - r1 + r2) * (d + r1 + r2)) / 2)

def findBlobs(img, scales=range(1, 10), threshold=30.0, max_overlap=0.05):
    from numpy import ones, triu, seterr
    old_errs = seterr(invalid='ignore')

    peaks = blobLOG(img, scales=scales, threshold=-threshold)
    radii = peaks[:, 0]
    positions = peaks[:, 1:]

    distances = np.linalg.norm(positions[:, None, :] - positions[None, :, :], axis=2)

    if positions.shape[1] == 2:
        intersections = circleIntersection(radii, radii.T, distances)
        volumes = np.pi * radii ** 2
    elif positions.shape[1] == 3:
        intersections = sphereIntersection(radii, radii.T, distances)
        volumes = 4/3 * np.pi * radii ** 3
    else:
        raise ValueError("Invalid dimensions for position ({}), need 2 or 3."
                         .format(positions.shape[1]))

    delete = ((intersections > (volumes * max_overlap))
              # Remove the smaller of the blobs
              & ((radii[:, None] < radii[None, :])
                 # Tie-break
                 | ((radii[:, None] == radii[None, :])
                    & triu(ones((len(peaks), len(peaks)), dtype='bool'))))
    ).any(axis=1)

    seterr(**old_errs)
    return peaks[~delete]


In [42]:
%matplotlib inline
import ipyvolume as ipv
import numpy as np
from scipy.ndimage.filters import convolve as conv
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
from skimage import feature

np.seterr(divide='ignore', invalid='ignore')

def generate_spheres_test_volume(size, num_beads):
    data = np.zeros((size, size, size))
    spheres = np.random.randint(5, size-5, size=(3, num_beads))
    for x in range(data.shape[0]):
        for y in range(data.shape[1]):
            for z in range(data.shape[2]):
                if np.any(np.linalg.norm(spheres.T-[x, y, z], axis=1) < 4.8):
                    data[x, y, z] = 1
                    
    return data

def deconv_rl(data, iterations, blur):
    estimate = 0.5 * np.ones_like(data)
    for i in range(iterations):
        print("It: {}".format(i))
        estimate = estimate * blur(data / blur(estimate))
    
    return estimate

def extract_area(data, point, size):
    data = np.pad(data, ((size, size), (size, size), (size, size)), mode='constant', constant_values=0)
    point = np.add(point, (size, size, size))
    return data[int(point[0]-size):int(point[0]+size), int(point[1]-size):int(point[1]+size), int(point[2]-size):int(point[2]+size)]

def gaussian_3d(x, A, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z):
    return (A * np.exp(-(
        (x[0]-mu_x)**2 / (2*sigma_x**2) +
        (x[1]-mu_y)**2 / (2*sigma_y**2) +
        (x[2]-mu_z)**2 / (2*sigma_z**2)
    )))

def fit_gaussian(data: np.ndarray):
    from scipy.optimize import curve_fit
    xs = np.arange(data.shape[0])
    ys = np.arange(data.shape[1])
    zs = np.arange(data.shape[2])
    xv, yv, zv = np.meshgrid(xs, ys, zs)
    xv = xv.ravel()
    yv = yv.ravel()
    zv = zv.ravel()
    xdata = np.vstack((xv, yv, zv))

    # start = [data.shape[0]//2, data.shape[1]//2, data.shape[2]//2, r, r, r]
    upper_bounds = [np.inf, data.shape[0], data.shape[1], data.shape[2], np.inf, np.inf, np.inf]
    p0 = (1, data.shape[0]//2, data.shape[1]//2, data.shape[2]//2, 1, 1, 1)
    popt, _ = curve_fit(gaussian_3d, xdata, data[xv, yv, zv], bounds=(np.zeros((7, )), upper_bounds), p0=p0)

    # print(popt[4:])
    return popt[4:]

STATIC_VOL = True

# psf = np.ones((10, 10, 10)) / 10**3
psf = np.zeros((13, 13, 13))
psf[6, 6, 6] = 1
psf = gaussian_filter(psf, (1, 1, 1))

# de psf

In [43]:
ipv.quickvolshow(psf)

VBox(children=(VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step=0.001), FloatSlider(value=0.5, max=1.0, step=0.001), FloatSlider(value=0.9, max=1.0, step=0.001))), HBox(children=(Label(value='opacities:'), FloatSlider(value=0.01, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001))))), Figure(ambient_coefficient=1.0, camera_center=[0.0, 0.0, 0.0], data_max=0.06349420361996902, diffuse_coefficient=0.0, height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], specular_coefficient=0.0, style={'background-color': 'white', 'box': {'visible': True}, 'axes': {'label': {'color': 'black'}, 'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}}}, tf=TransferFunctionWidgetJs3(level3=0.9), volume_data=array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 2.39699161e-12, ...,
         2.39699161e-12, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 2.39699161e-12, ...,
         2.39699161e-12, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       ...,

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 2.39699161e-12, ...,
         2.39699161e-12, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 2.39699161e-12, ...,
         2.39699161e-12, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e

In [44]:
vol = generate_spheres_test_volume(55, 12)
vol = np.pad(vol, 9, 'constant', constant_values=0)
vol[5, 5, 5] = 1

# het gegenereerde input volume

In [45]:
ipv.quickvolshow(vol)

VBox(children=(VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step=0.001), FloatSlider(value=0.5, max=1.0, step=0.001), FloatSlider(value=0.9, max=1.0, step=0.001))), HBox(children=(Label(value='opacities:'), FloatSlider(value=0.01, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001))))), Figure(ambient_coefficient=1.0, camera_center=[0.0, 0.0, 0.0], data_max=1.0, diffuse_coefficient=0.0, height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], specular_coefficient=0.0, style={'background-color': 'white', 'box': {'visible': True}, 'axes': {'label': {'color': 'black'}, 'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}}}, tf=TransferFunctionWidgetJs3(level3=0.9), volume_data=array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]]), width=400, xlim=[0.0, 1.0], ylim=[0.0, 1.0], zlim=[0.0, 1.0])))

In [46]:
blurred_vol = conv(vol, psf)

# het geblurrede volume

In [47]:
ipv.quickvolshow(blurred_vol)

VBox(children=(VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step=0.001), FloatSlider(value=0.5, max=1.0, step=0.001), FloatSlider(value=0.9, max=1.0, step=0.001))), HBox(children=(Label(value='opacities:'), FloatSlider(value=0.01, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001))))), Figure(ambient_coefficient=1.0, camera_center=[0.0, 0.0, 0.0], data_max=0.9999820324351394, diffuse_coefficient=0.0, height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], specular_coefficient=0.0, style={'background-color': 'white', 'box': {'visible': True}, 'axes': {'label': {'color': 'black'}, 'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}}}, tf=TransferFunctionWidgetJs3(level3=0.9), volume_data=array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.39699161e-12, 7.93774607e-11, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 7.93774607e-11, 2.62862048e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 7.93774607e-11, 2.62862048e-09, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.62862048e-09, 8.70479554e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       ...,

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+

# De bead detecteren en de sigma estimeren

In [48]:
blobs = findBlobs(blurred_vol[:10, :10, :10], threshold=0.01)

In [49]:
blob_area = extract_area(blurred_vol, blobs[0][1:], 5)
est_sigma = fit_gaussian(blob_area)

In [50]:
noisy_blurred_vol = blurred_vol + (np.random.poisson(lam=25, size=blurred_vol.shape) - 10) / 255.

# Het volume met noise

In [51]:
ipv.quickvolshow(noisy_blurred_vol, level=[0.17, 0.50, 0.90], opacity=[0.01, 0.05, 0.1], data_min=0, data_max=1)

VBox(children=(VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.17, max=1.0, step=0.001), FloatSlider(value=0.5, max=1.0, step=0.001), FloatSlider(value=0.9, max=1.0, step=0.001))), HBox(children=(Label(value='opacities:'), FloatSlider(value=0.01, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001))))), Figure(ambient_coefficient=1.0, camera_center=[0.0, 0.0, 0.0], data_max=1.0, diffuse_coefficient=0.0, height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], specular_coefficient=0.0, style={'background-color': 'white', 'box': {'visible': True}, 'axes': {'label': {'color': 'black'}, 'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}}}, tf=TransferFunctionWidgetJs3(level1=0.17, level3=0.9), volume_data=array([[[0.07843137, 0.03529412, 0.06666667, ..., 0.0745098 ,
         0.07058824, 0.04705882],
        [0.05882353, 0.0745098 , 0.04705882, ..., 0.06666667,
         0.04705882, 0.05490196],
        [0.05490196, 0.07843137, 0.04705882, ..., 0.03529412,
         0.0745098 , 0.05882353],
        ...,
        [0.01568627, 0.10588235, 0.05098039, ..., 0.07058824,
         0.05098039, 0.05882353],
        [0.05490196, 0.07843137, 0.03921569, ..., 0.05490196,
         0.04313725, 0.05098039],
        [0.04313725, 0.04313725, 0.04313725, ..., 0.0745098 ,
         0.05098039, 0.0627451 ]],

       [[0.03529412, 0.04705882, 0.05098039, ..., 0.06666667,
         0.07058824, 0.0627451 ],
        [0.05098039, 0.10196078, 0.04705882, ..., 0.02352941,
         0.04705882, 0.07843137],
        [0.04313725, 0.08627451, 0.09411765, ..., 0.04313725,
         0.05098039, 0.05882353],
        ...,
        [0.03921569, 0.07843137, 0.09019608, ..., 0.09411765,
         0.05098039, 0.05098039],
        [0.09411765, 0.00784314, 0.07843137, ..., 0.0627451 ,
         0.0745098 , 0.03529412],
        [0.02745098, 0.08235294, 0.04705882, ..., 0.05882353,
         0.03137255, 0.04705882]],

       [[0.03529412, 0.07843137, 0.08235294, ..., 0.0745098 ,
         0.01960784, 0.07058824],
        [0.08627451, 0.09411765, 0.07450981, ..., 0.0745098 ,
         0.05490196, 0.09019608],
        [0.09803922, 0.06666667, 0.05490205, ..., 0.0627451 ,
         0.05490196, 0.05490196],
        ...,
        [0.05098039, 0.02352941, 0.09019608, ..., 0.04313725,
         0.0627451 , 0.08235294],
        [0.0627451 , 0.0745098 , 0.05882353, ..., 0.09803922,
         0.02352941, 0.04705882],
        [0.0745098 , 0.0627451 , 0.01568627, ..., 0.07843137,
         0.03137255, 0.05490196]],

       ...,

       [[0.05098039, 0.06666667, 0.0745098 , ..., 0.07058824,
         0.07843137, 0.04705882],
        [0.05490196, 0.08627451, 0.0745098 , ..., 0.0627451 ,
         0.03137255, 0.04705882],
        [0.0745098 , 0.06666667, 0.03921569, ..., 0.07843137,
         0.07058824, 0.05098039],
        ...,
        [0.0627451 , 0.10196078, 0.0745098 , ..., 0.0627451 ,
         0.05098039, 0.06666667],
        [0.03137255, 0.05098039, 0.05490196, ..., 0.06666667,
         0.07058824, 0.04313725],
        [0.05882353, 0.05098039, 0.00784314, ..., 0.0627451 ,
         0.03921569, 0.05490196]],

       [[0.04705882, 0.06666667, 0.05098039, ..., 0.07843137,
         0.02745098, 0.04313725],
        [0.05098039, 0.05490196, 0.06666667, ..., 0.06666667,
         0.04705882, 0.03137255],
        [0.07843137, 0.0627451 , 0.05490196, ..., 0.03529412,
         0.05490196, 0.0627451 ],
        ...,
        [0.03921569, 0.05098039, 0.05098039, ..., 0.09019608,
         0.05490196, 0.04313725],
        [0.07843137, 0.07058824, 0.05098039, ..., 0.09019608,
         0.07058824, 0.0627451 ],
        [0.05098039, 0.05490196, 0.03137255, ..., 0.02352941,
         0.05882353, 0.05882353]],

       [[0.01568627, 0.07843137, 0.02745098, ..., 0.02352941,
       

# Deconvolven met de gestimeerde sigma

In [55]:
blur = lambda x: gaussian_filter(x, est_sigma)
result = deconv_rl(noisy_blurred_vol, 22, blur)
result = result.clip(-0.5, 1.18)
result = (result-result.min())/(result.max()-result.min())

It: 0
It: 1
It: 2
It: 3
It: 4
It: 5
It: 6
It: 7
It: 8
It: 9
It: 10
It: 11
It: 12
It: 13
It: 14
It: 15
It: 16
It: 17
It: 18
It: 19
It: 20
It: 21


In [56]:
ipv.quickvolshow(result, level=[0.17, 0.50, 0.90], opacity=[0.01, 0.05, 0.1], data_min=0, data_max=1)

VBox(children=(VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.17, max=1.0, step=0.001), FloatSlider(value=0.5, max=1.0, step=0.001), FloatSlider(value=0.9, max=1.0, step=0.001))), HBox(children=(Label(value='opacities:'), FloatSlider(value=0.01, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001))))), Figure(ambient_coefficient=1.0, camera_center=[0.0, 0.0, 0.0], data_max=1.0, diffuse_coefficient=0.0, height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], specular_coefficient=0.0, style={'background-color': 'white', 'box': {'visible': True}, 'axes': {'label': {'color': 'black'}, 'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}}}, tf=TransferFunctionWidgetJs3(level1=0.17, level3=0.9), volume_data=array([[[0.05742697, 0.02531964, 0.02965402, ..., 0.07931237,
         0.05423722, 0.02811588],
        [0.04909621, 0.06041755, 0.06355621, ..., 0.05268316,
         0.03069424, 0.03907873],
        [0.03103476, 0.0545343 , 0.05495357, ..., 0.02227793,
         0.02389758, 0.08041807],
        ...,
        [0.01886736, 0.09391226, 0.06752214, ..., 0.08031034,
         0.0413344 , 0.03061072],
        [0.04018065, 0.06843913, 0.03447115, ..., 0.06199881,
         0.03874363, 0.03338643],
        [0.02105746, 0.0258628 , 0.02939284, ..., 0.06664505,
         0.05217011, 0.06256954]],

       [[0.01660089, 0.03071348, 0.02869581, ..., 0.05312613,
         0.04187457, 0.07054965],
        [0.0447854 , 0.08839994, 0.05966164, ..., 0.03394329,
         0.02902053, 0.08216602],
        [0.05565162, 0.07829436, 0.04558639, ..., 0.02484357,
         0.01949409, 0.05283223],
        ...,
        [0.0286195 , 0.06707368, 0.07356701, ..., 0.07939437,
         0.04531395, 0.03415796],
        [0.04594888, 0.046843  , 0.04463857, ..., 0.07156574,
         0.034376  , 0.02149651],
        [0.02837567, 0.02597583, 0.03034453, ..., 0.03910038,
         0.02215508, 0.01980882]],

       [[0.01957549, 0.07361878, 0.05965174, ..., 0.04940002,
         0.02112648, 0.05073827],
        [0.06118183, 0.1083379 , 0.0746732 , ..., 0.04811747,
         0.03324795, 0.08901627],
        [0.09275193, 0.06841982, 0.03464591, ..., 0.06433254,
         0.03591614, 0.04783403],
        ...,
        [0.0335497 , 0.04296578, 0.10331697, ..., 0.05660812,
         0.05810683, 0.06801381],
        [0.06564172, 0.03870461, 0.06126981, ..., 0.07393399,
         0.04012095, 0.03767944],
        [0.08638354, 0.02977853, 0.02092338, ..., 0.07423256,
         0.02181499, 0.0157464 ]],

       ...,

       [[0.01902826, 0.07004577, 0.06792196, ..., 0.0995624 ,
         0.04948633, 0.02089292],
        [0.04557692, 0.08900492, 0.0946962 , ..., 0.06355506,
         0.04371568, 0.02908883],
        [0.07386798, 0.06745418, 0.05423381, ..., 0.06229135,
         0.05914681, 0.04331953],
        ...,
        [0.05079581, 0.0771767 , 0.04455385, ..., 0.09884527,
         0.08644083, 0.05425564],
        [0.03535145, 0.03797166, 0.0221533 , ..., 0.07568951,
         0.05419222, 0.047134  ],
        [0.04873434, 0.01998566, 0.01134089, ..., 0.02738979,
         0.01522255, 0.01828061]],

       [[0.01823676, 0.06975024, 0.04852748, ..., 0.05108163,
         0.02900018, 0.02161986],
        [0.06603989, 0.0709081 , 0.03703352, ..., 0.04543298,
         0.03345051, 0.02402615],
        [0.08177297, 0.05032028, 0.02585335, ..., 0.04331496,
         0.05190772, 0.03609535],
        ...,
        [0.0328112 , 0.05622135, 0.05910035, ..., 0.06059446,
         0.0620368 , 0.04235948],
        [0.05704127, 0.0523684 , 0.03505683, ..., 0.05612696,
         0.07297165, 0.05152466],
        [0.06039596, 0.0215246 , 0.01495214, ..., 0.02455115,
         0.03974376, 0.04331525]],

       [[0.00886606, 0.03679485, 0.01401689, ..., 0.00991225,
       

# Het verschil tussen het origineel en het gedeconvolvede resultaat

In [57]:
diff = np.abs(vol - result)

In [58]:
ipv.quickvolshow(diff, level=[0.17, 0.50, 0.90], opacity=[0.01, 0.05, 0.1], data_min=0, data_max=1)

VBox(children=(VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.17, max=1.0, step=0.001), FloatSlider(value=0.5, max=1.0, step=0.001), FloatSlider(value=0.9, max=1.0, step=0.001))), HBox(children=(Label(value='opacities:'), FloatSlider(value=0.01, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001))))), Figure(ambient_coefficient=1.0, camera_center=[0.0, 0.0, 0.0], data_max=1.0, diffuse_coefficient=0.0, height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], specular_coefficient=0.0, style={'background-color': 'white', 'box': {'visible': True}, 'axes': {'label': {'color': 'black'}, 'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}}}, tf=TransferFunctionWidgetJs3(level1=0.17, level3=0.9), volume_data=array([[[0.05742697, 0.02531964, 0.02965402, ..., 0.07931237,
         0.05423722, 0.02811588],
        [0.04909621, 0.06041755, 0.06355621, ..., 0.05268316,
         0.03069424, 0.03907873],
        [0.03103476, 0.0545343 , 0.05495357, ..., 0.02227793,
         0.02389758, 0.08041807],
        ...,
        [0.01886736, 0.09391226, 0.06752214, ..., 0.08031034,
         0.0413344 , 0.03061072],
        [0.04018065, 0.06843913, 0.03447115, ..., 0.06199881,
         0.03874363, 0.03338643],
        [0.02105746, 0.0258628 , 0.02939284, ..., 0.06664505,
         0.05217011, 0.06256954]],

       [[0.01660089, 0.03071348, 0.02869581, ..., 0.05312613,
         0.04187457, 0.07054965],
        [0.0447854 , 0.08839994, 0.05966164, ..., 0.03394329,
         0.02902053, 0.08216602],
        [0.05565162, 0.07829436, 0.04558639, ..., 0.02484357,
         0.01949409, 0.05283223],
        ...,
        [0.0286195 , 0.06707368, 0.07356701, ..., 0.07939437,
         0.04531395, 0.03415796],
        [0.04594888, 0.046843  , 0.04463857, ..., 0.07156574,
         0.034376  , 0.02149651],
        [0.02837567, 0.02597583, 0.03034453, ..., 0.03910038,
         0.02215508, 0.01980882]],

       [[0.01957549, 0.07361878, 0.05965174, ..., 0.04940002,
         0.02112648, 0.05073827],
        [0.06118183, 0.1083379 , 0.0746732 , ..., 0.04811747,
         0.03324795, 0.08901627],
        [0.09275193, 0.06841982, 0.03464591, ..., 0.06433254,
         0.03591614, 0.04783403],
        ...,
        [0.0335497 , 0.04296578, 0.10331697, ..., 0.05660812,
         0.05810683, 0.06801381],
        [0.06564172, 0.03870461, 0.06126981, ..., 0.07393399,
         0.04012095, 0.03767944],
        [0.08638354, 0.02977853, 0.02092338, ..., 0.07423256,
         0.02181499, 0.0157464 ]],

       ...,

       [[0.01902826, 0.07004577, 0.06792196, ..., 0.0995624 ,
         0.04948633, 0.02089292],
        [0.04557692, 0.08900492, 0.0946962 , ..., 0.06355506,
         0.04371568, 0.02908883],
        [0.07386798, 0.06745418, 0.05423381, ..., 0.06229135,
         0.05914681, 0.04331953],
        ...,
        [0.05079581, 0.0771767 , 0.04455385, ..., 0.09884527,
         0.08644083, 0.05425564],
        [0.03535145, 0.03797166, 0.0221533 , ..., 0.07568951,
         0.05419222, 0.047134  ],
        [0.04873434, 0.01998566, 0.01134089, ..., 0.02738979,
         0.01522255, 0.01828061]],

       [[0.01823676, 0.06975024, 0.04852748, ..., 0.05108163,
         0.02900018, 0.02161986],
        [0.06603989, 0.0709081 , 0.03703352, ..., 0.04543298,
         0.03345051, 0.02402615],
        [0.08177297, 0.05032028, 0.02585335, ..., 0.04331496,
         0.05190772, 0.03609535],
        ...,
        [0.0328112 , 0.05622135, 0.05910035, ..., 0.06059446,
         0.0620368 , 0.04235948],
        [0.05704127, 0.0523684 , 0.03505683, ..., 0.05612696,
         0.07297165, 0.05152466],
        [0.06039596, 0.0215246 , 0.01495214, ..., 0.02455115,
         0.03974376, 0.04331525]],

       [[0.00886606, 0.03679485, 0.01401689, ..., 0.00991225,
       